Set-Up

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..')))

from scripts import request, download, extraction, stats
from scripts.constants import Constants

project_path = os.path.abspath(os.path.expandvars(Constants.PROJECT_PATH))
experiments_path = os.path.join(project_path, Constants.Directories.EXPERIMENTS)

connection_info = request.extract_infos('../private.json')

Download Articles

In [ ]:
article_dir = "CS_GroundTruth"
articles_path = os.path.join(experiments_path, Constants.Directories.ARTICLES, article_dir)

tables_path = os.path.join(experiments_path, Constants.Directories.EXTRACTED_TABLE)
json_file_name = f"{article_dir}.json"

In [ ]:
# Download articles as HTML files
search_query = '"Entity Matching"'
download.get_articles(search_query, articles_path)

In [ ]:
extraction.extract_and_save_tables(articles_path, tables_path, json_file_name)

Claim Extraction

In [ ]:
output_dir = "cs_label_html"
tables_file_name = 'CS_GroundTruth.json'
msgs_dir = "CS_Table_Labeling"

In [ ]:
request.set_up_test_dir(project_path, output_dir, tables_file_name, msgs_dir)
test_info = request.get_test_info(project_path, output_dir)

In [ ]:
request.run_test(connection_info, test_info, num_thread=50, max_cycles=3)

Ground Truth

In [ ]:
gt_path = os.path.join(experiments_path, Constants.Directories.GROUND_TRUTH)
gt_file = os.path.join(gt_path, "table_labeled.ods")
gt_answer_path = os.path.join(gt_path, "labeling")

stats.write_ground_truth(gt_file, gt_answer_path)

output_dir = os.path.join(experiments_path, Constants.Directories.OUTPUT)
caption_dir = os.path.join(output_dir, "cs_label_caption")
html_dir = os.path.join(output_dir, "cs_label_html")

output_dirs = [
    (caption_dir, 1),
    (html_dir, 1)
]

json_path = os.path.join(experiments_path, "comparison.json")
stats.compare_results(gt_answer_path, output_dirs, json_path)